In [3]:
import sys
import os
import pandas as pd
import pynance as pn
import talib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [2]:
# ---
# title: Quantitative Analysis for 2020 AAPL Stock Data
# description: Notebook to load, prepare, and analyze AAPL stock data using pynance and TaLib
# ---

# ### 1.Adjust Python Path

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Set plot style
sns.set_style("whitegrid")

# ### 2. Load and Prepare Stock Data
stock_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
stock_df = pd.read_csv('../data/stock_prices_cleaned_2020.csv')
for col in stock_columns:
    if col not in stock_df.columns:
        raise ValueError(f"Missing column: {col}")

# Convert Date to datetime and set as index
stock_df['Date'] = pd.to_datetime(stock_df['Date'], utc=True)
stock_df.set_index('Date', inplace=True)

# Ensure numeric data types
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
stock_df[numeric_columns] = stock_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Drop rows with missing values
stock_df.dropna(subset=numeric_columns, inplace=True)

# Verify data
print("=== Stock Data Info ===")
stock_df.info()
print("\n=== Stock Data Sample ===")
display(stock_df.head())

# ### 3. Basic Quantitative Analysis with Pynance
# Calculate daily returns
stock_df['Daily_Return'] = pn.data.returns(stock_df['Close'])

# Calculate moving averages
stock_df['SMA_20'] = pn.data.sma(stock_df['Close'], window=20)
stock_df['EMA_20'] = pn.data.ema(stock_df['Close'], window=20)

# Calculate volatility
stock_df['Volatility_20'] = stock_df['Daily_Return'].rolling(window=20).std() * (252 ** 0.5)  # Annualized

# Display summary statistics
print("\n=== Summary Statistics ===")
display(stock_df[['Close', 'Daily_Return', 'SMA_20', 'EMA_20', 'Volatility_20']].describe())

# Visualize price and moving averages
plt.figure(figsize=(12, 6))
plt.plot(stock_df.index, stock_df['Close'], label='Close Price', color='blue')
plt.plot(stock_df.index, stock_df['SMA_20'], label='20-Day SMA', color='orange')
plt.plot(stock_df.index, stock_df['EMA_20'], label='20-Day EMA', color='green')
plt.title('AAPL Stock Price and Moving Averages (2020)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/aapl_price_moving_averages_2020.png')
plt.show()

# Visualize volatility
plt.figure(figsize=(12, 6))
plt.plot(stock_df.index, stock_df['Volatility_20'], color='red')
plt.title('AAPL 20-Day Annualized Volatility (2020)')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/aapl_volatility_2020.png')
plt.show()

# ### 4. Technical Indicators with TaLib
# Calculate RSI
stock_df['RSI_14'] = talib.RSI(stock_df['Close'], timeperiod=14)

# Calculate MACD
stock_df['MACD'], stock_df['MACD_Signal'], stock_df['MACD_Hist'] = talib.MACD(
    stock_df['Close'], fastperiod=12, slowperiod=26, signalperiod=9
)

# Calculate Bollinger Bands
stock_df['BB_Upper'], stock_df['BB_Middle'], stock_df['BB_Lower'] = talib.BBANDS(
    stock_df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2
)

# Display technical indicators
print("\n=== Technical Indicators Sample ===")
display(stock_df[['Close', 'RSI_14', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Lower']].tail())

# Visualize RSI
plt.figure(figsize=(12, 6))
plt.plot(stock_df.index, stock_df['RSI_14'], color='purple')
plt.axhline(70, color='red', linestyle='--', alpha=0.5, label='Overbought (70)')
plt.axhline(30, color='green', linestyle='--', alpha=0.5, label='Oversold (30)')
plt.title('AAPL 14-Day RSI (2020)')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/aapl_rsi_2020.png')
plt.show()

# Visualize MACD
plt.figure(figsize=(12, 6))
plt.plot(stock_df.index, stock_df['MACD'], label='MACD', color='blue')
plt.plot(stock_df.index, stock_df['MACD_Signal'], label='Signal Line', color='orange')
plt.bar(stock_df.index, stock_df['MACD_Hist'], color='gray', alpha=0.3, label='Histogram')
plt.title('AAPL MACD (2020)')
plt.xlabel('Date')
plt.ylabel('MACD')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/aapl_macd_2020.png')
plt.show()

# Visualize Bollinger Bands
plt.figure(figsize=(12, 6))
plt.plot(stock_df.index, stock_df['Close'], label='Close Price', color='blue')
plt.plot(stock_df.index, stock_df['BB_Upper'], label='Upper Band', color='red', linestyle='--')
plt.plot(stock_df.index, stock_df['BB_Lower'], label='Lower Band', color='green', linestyle='--')
plt.plot(stock_df.index, stock_df['BB_Middle'], label='Middle Band', color='orange')
plt.title('AAPL Bollinger Bands (2020)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/aapl_bollinger_bands_2020.png')
plt.show()

# ### 5. Save Results
stock_df.to_csv('../data/aapl_quantitative_analysis_2020.csv')

=== Stock Data Info ===
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2020-01-02 00:00:00+00:00 to 2020-12-31 00:00:00+00:00
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          253 non-null    float64
 1   High          253 non-null    float64
 2   Low           253 non-null    float64
 3   Close         253 non-null    float64
 4   Adj Close     253 non-null    float64
 5   Volume        253 non-null    int64  
 6   Dividends     253 non-null    float64
 7   Stock Splits  253 non-null    float64
 8   stock         253 non-null    object 
 9   date_only     253 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 21.7+ KB

=== Stock Data Sample ===


,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,stock,date_only
Date,,,,,,,,,,
2020-01-02 00:00:00+00:00,74.059998,75.150002,73.797501,75.087502,72.876099,135480400,0.0,0.0,AAPL,2020-01-02
2020-01-03 00:00:00+00:00,74.287498,75.144997,74.125000,74.357498,72.167595,146322800,0.0,0.0,AAPL,2020-01-03
2020-01-06 00:00:00+00:00,73.447502,74.989998,73.187500,74.949997,72.742661,118387200,0.0,0.0,AAPL,2020-01-06
2020-01-07 00:00:00+00:00,74.959999,75.224998,74.370003,74.597504,72.400536,108872000,0.0,0.0,AAPL,2020-01-07
2020-01-08 00:00:00+00:00,74.290001,76.110001,74.290001,75.797501,73.565208,132079200,0.0,0.0,AAPL,2020-01-08


AttributeError: module 'pynance.data' has no attribute 'returns'